In [31]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../artifactory/')

In [32]:
import torch
import pickle
import warnings
import numpy as np
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.utils.data import DataLoader
from pathlib import Path
from itertools import repeat
from artifact import Saw
from data import ArtifactDataset, CachedArtifactDataset
from detector import ConvolutionDetector, WindowTransformerDetector
from utilities import parameters_k
import wandb
from datetime import datetime
import pytz

# stop warnings
torch.set_float32_matmul_precision("high")
warnings.filterwarnings("ignore", ".*does not have many workers.*")

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [34]:
# Storing hyperparameters as a dictionary, because we can directly log this config dict to W&B.
CONFIG = dict(
    # width of window
    width = 512,
    convolution_features=[256, 128, 64, 32],
    convolution_width=[5, 9, 17, 33],
    convolution_dropout=0.0,
    transformer_heads=2,
    transformer_feedforward=128,
    transformer_layers=2,
    transformer_dropout=0,
    loss="mask",
    loss_boost_fp=0,
    
    artifact=Saw(min_width=4, max_width=32),
    # Optimizer Parameter

    # LearningRate Scheduler
    
    # parameters for study
    batch_size = 32, # 'values': [32, 64, 128]
    
    wandb_group_name = "test_setup",
    wandb_project_name = "artifactory"
)

All settings.

In [36]:
# model
model = WindowTransformerDetector(window=CONFIG["width"],                    
                                  convolution_features=CONFIG["convolution_features"],
                                  convolution_width=CONFIG["convolution_width"],
                                  convolution_dropout=CONFIG["convolution_dropout"],
                                  transformer_heads=CONFIG["transformer_heads"],
                                  transformer_feedforward=CONFIG["transformer_feedforward"],
                                  transformer_layers=CONFIG["transformer_layers"],
                                  transformer_dropout=CONFIG["transformer_dropout"],
                                  loss=CONFIG["loss"],
                                  loss_boost_fp=CONFIG["loss_boost_fp"])
# model = ConvolutionDetector(convolution_features=[128, 64, 32],
#                             convolution_width=[5, 9, 33],
#                             convolution_dilation=[1, 1, 1],
#                             convolution_dropout=0.0,
#                             convolution_activation="sigmoid")
model_name = f"{model.__class__.__name__}_{parameters_k(model)}_{datetime.now(pytz.timezone('Europe/Amsterdam')).strftime('%d-%m-%Y_%H:%M:%S')}"
CONFIG['wandb_run_name'] = model_name

val_file = Path(f"../data/validation{CONFIG['width']}.all.pkl")
val_datasets = [
    #"australian_electricity_demand_dataset",
    #"electricity_hourly_dataset",
    "electricity_load_diagrams",
    #"HouseholdPowerConsumption1",
    #"HouseholdPowerConsumption2",
    #"london_smart_meters_dataset_without_missing_values",
    #"solar_10_minutes_dataset",
    #"wind_farms_minutely_dataset_without_missing_values",
]
train_datasets = [
    #"australian_electricity_demand_dataset",
    #"electricity_hourly_dataset",
    "electricity_load_diagrams",
    #"HouseholdPowerConsumption1",
    #"HouseholdPowerConsumption2",
    #"london_smart_meters_dataset_without_missing_values",
    #"solar_10_minutes_dataset",
    #"wind_farms_minutely_dataset_without_missing_values",
]
print(model_name)

WindowTransformerDetector_528.96K_18-12-2023_16:34:18


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


Loading data.

In [37]:
def load_series(names: list[str], split: str):
    series = list()
    counts = list()
    for name in names:
        with open(f"../data/processed/{name}_{split}.pickle", "rb") as f:
            raw = [a for a in pickle.load(f) if len(a) > CONFIG["width"]]
            series.extend(np.array(a).astype(np.float32) for a in raw)
            counts.extend(repeat(1 / len(raw), len(raw)))
    counts = np.array(counts)
    return series, counts / counts.sum()

In [38]:
# train
train_data, train_weights = load_series(train_datasets, "TRAIN")
train_dataset = ArtifactDataset(train_data,
                                CONFIG["artifact"],
                                width=CONFIG["width"],
                                padding=64,
                                weight=train_weights) 
train_loader = DataLoader(train_dataset, batch_size=CONFIG["batch_size"])

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [39]:
# validation
if not val_file.exists():
    val_data, val_weights = load_series(val_datasets, "TEST")
    val_gen = ArtifactDataset(val_data,
                              CONFIG["artifact"],
                              width=CONFIG["width"],
                              padding=64,
                              weight=val_weights)
    val = CachedArtifactDataset.generate(val_gen,
                                         n=2048,
                                         to=val_file)
else:
    val = CachedArtifactDataset(file=val_file)
val_loader = DataLoader(val, batch_size=CONFIG["batch_size"])

Sanity check.

In [40]:
batch = next(iter(train_loader))
batch["data"]

tensor([[0.4218, 0.4115, 0.4424,  ..., 0.2263, 0.4321, 0.3807],
        [0.4962, 0.4962, 0.5077,  ..., 0.5231, 0.5269, 0.5346],
        [0.6597, 0.5825, 0.5357,  ..., 0.2591, 0.2871, 0.3747],
        ...,
        [0.1793, 0.2249, 0.2006,  ..., 0.4357, 0.4204, 0.4143],
        [0.0846, 0.0923, 0.1000,  ..., 0.0615, 0.0769, 0.0769],
        [0.0000, 0.0000, 0.0000,  ..., 0.1379, 0.1724, 0.1379]])

In [41]:
model(batch["data"]).shape

torch.Size([32, 512])

Training!

In [45]:
# Initialize W&B run
run = wandb.init(project=CONFIG["wandb_project_name"], 
        config=CONFIG,
        entity="hvonhue",
        group=CONFIG["wandb_group_name"], 
        job_type='train',
        name=CONFIG["wandb_run_name"])

wandb.config.type = 'baseline'

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁
grad_2.0_norm/convolutions.0.bias,▁
grad_2.0_norm/convolutions.0.weight,▁
grad_2.0_norm/convolutions.2.bias,▁
grad_2.0_norm/convolutions.2.weight,▁
grad_2.0_norm/convolutions.4.bias,▁
grad_2.0_norm/convolutions.4.weight,▁
grad_2.0_norm/convolutions.6.bias,▁
grad_2.0_norm/convolutions.6.weight,▁
grad_2.0_norm/linear.bias,▁
grad_2.0_norm/linear.weight,▁


In [46]:
# initialize callbacks
checkpointcallback = ModelCheckpoint(monitor="validation",
                                     mode="min",
                                     save_top_k=1)
lr_monitor = LearningRateMonitor(logging_interval='step')

# initialize logger
logger = WandbLogger(project="artifactory",
                     name=model_name,
                     log_model="all")

# initialize trainer
trainer = Trainer(logger=logger,
                  max_steps=50000,
                  val_check_interval=1000,
                  callbacks=[checkpointcallback,
                             lr_monitor])

# train
trainer.fit(model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name         | Type                        | Params
-------------------------------------------------------------
0 | convolutions | Sequential                  | 503 K 
1 | position     | SinusoidalPositionEmbedding | 0     
2 | dropout      | Dropout                     | 0     
3 | transformer  | TransformerEncoder          | 25.4 K
4 | linear       | Linear                      | 33    
-------------------------------------------------------------
528 K     Trainable params
0         Non-trainable params
528 K     Total params
2.116     Total estimated model params siz

Epoch 0: |          | 1505/? [32:32<00:00,  0.77it/s, v_num=b0l7]          

In [ ]:
# End Wandb run
run.finish()